In [3]:
# Dimensiunile matricei
HEIGHT, WIDTH = 6, 7

# Pozițiile din tuplul ce constituie o stare
BOARD, NEXT_PLAYER = 0, 1

# Jucătorii
RED, BLUE = 1, 2
name = ["", "ROȘU", "ALBASTRU", "REMIZĂ"]

# Funcție ce întoarce o stare inițială
def init_state():
    return ([[0 for row in range(HEIGHT)] for col in range(WIDTH)], RED)

# Funcție ce afișează o stare
def print_state(state):
    for row in range(HEIGHT - 1, -1, -1):
        ch = " RA"
        l = map(lambda col: ch[state[BOARD][col][row]], range(WIDTH))
        print("|" + "".join(l) + "|")
    print("+" + "".join("-" * WIDTH) + "+")
    print(f"Urmează: {state[NEXT_PLAYER]} - {name[state[NEXT_PLAYER]]}")

# Se afișează starea inițială a jocului
print("Starea inițială:")
print_state(init_state())

Starea inițială:
|       |
|       |
|       |
|       |
|       |
|       |
+-------+
Urmează: 1 - ROȘU


In [4]:
# Funcție ce întoarce acțiunile valide dintr-o stare dată
def get_available_actions(state):
    moves = []

    for i in range(len(state[BOARD])):
        if any(cell == 0 for cell in state[BOARD][i]):
            moves.append(i)

    return moves


from copy import deepcopy
from functools import reduce

# Funcție ce întoarce starea în care se ajunge prin aplicarea unei acțiuni
def apply_action(state, action):
    new_board = deepcopy(state[BOARD])
    new_board[action][new_board[action].index(0,0)] = state[NEXT_PLAYER]
    return (new_board, 3 - state[NEXT_PLAYER])


# Se afișează starea la care se ajunge prin aplicarea unor acțiuni
somestate = reduce(apply_action, [3, 4, 3, 2, 2, 6, 3, 3, 3, 3], init_state())
print_state(somestate)
get_available_actions(somestate)

|   A   |
|   R   |
|   A   |
|   R   |
|  RR   |
|  ARA A|
+-------+
Urmează: 1 - ROȘU


[0, 1, 2, 4, 5, 6]

In [5]:
# Funcție ce verifică dacă o stare este finală
def is_final(state):
    # Verificăm dacă matricea este plină
    if not any([0 in col for col in state[BOARD]]): return 3
    # Jucătorul care doar ce a mutat ar putea să fie câștigător
    player = 3 - state[NEXT_PLAYER]
    
    ok = lambda pos: all([state[BOARD][c][r] == player for (r, c) in pos])
    # Verificăm verticale
    for row in range(HEIGHT):
        for col in range(WIDTH - 4):
            if ok([(row, col + i) for i in range(4)]): return player
    # Verificăm orizontale
    for col in range(WIDTH):
        for row in range(HEIGHT - 4):
            if ok([(row + i, col) for i in range(4)]): return player
    # Verificăm diagonale
    for col in range(WIDTH - 4):
        for row in range(HEIGHT - 4):
            if ok([(row + i, col + i) for i in range(4)]): return player
    for col in range(WIDTH-4):
        for row in range(HEIGHT - 4):
            if ok([(row + i, col + 4 - i) for i in range(4)]): return player
    return False

In [6]:
# Afișăm o stare finală oarecare
from random import choice

rand_state = init_state()
while not is_final(rand_state):
    actions = get_available_actions(rand_state)
    if not actions:
        break
    action = choice(get_available_actions(rand_state))
    rand_state = apply_action(rand_state, action)

print_state(rand_state)
print(f"Învingător: {name[is_final(rand_state)]}")

|       |
|       |
|  ARAR |
|RARRRAA|
|ARAARRR|
|RRAAARA|
+-------+
Urmează: 2 - ALBASTRU
Învingător: ROȘU


In [7]:
# Exemplu: Se afișează starea obținută prin aplicarea unor acțiuni
all_actions = [1, 2, 1, 3, 1, 4, 2, 5]
some_state = reduce(apply_action, all_actions, init_state())
print_state(some_state)
print(f"Învingător: {name[is_final(some_state)]}")

|       |
|       |
|       |
| R     |
| RR    |
| RAAAA |
+-------+
Urmează: 1 - ROȘU
Învingător: ALBASTRU


In [10]:
# Constante

N = 'N'
Q = 'Q'
PARENT = 'parent'
ACTIONS = 'actions'

In [11]:
def print_tree(tree, indent = 0):
    if not tree:
        return
    tab = "".join(" " * indent)
    print(f"{tab}N = {tree[N]}, Q = {tree[Q]}")
    for a in tree[ACTIONS]:
        print(f"{tab} {a} => ")
        print_tree(tree[ACTIONS][a], indent + 3)


def number_of_nodes(tree):
    if not tree:
        return 0
    no = 1
    for child in tree[ACTIONS].values():
        no += number_of_nodes(child)
    return no

In [12]:
# Funcție ce întoarce un nod nou,
# eventual copilul unui nod dat ca argument
def init_node(parent = None):
    return {N: 0, Q: 0, PARENT: parent, ACTIONS: {}}

In [13]:
from math import sqrt, log
import numpy as np

CP = 1.0 / sqrt(2.0)

# Funcție ce alege o acțiune dintr-un nod
def select_action(node, c = CP):
    """
    Se caută acțiunea a care maximizează expresia:
    Q_a / N_a  +  c * sqrt(2 * log(N_node) / N_a)
    """
    N_node = node[N]
    max_score = -1
    best_action = None

    for a, n in node[ACTIONS].items():
        crt_score = n[Q] / n[N] + c * sqrt(2 * log(N_node) / n[N])

        if max_score < crt_score:
            max_score = crt_score
            best_action = a

    return best_action

# Scurtă testare
test_root = {N: 6, Q: 0.75, PARENT: None, ACTIONS: {}}
test_root[ACTIONS][3] = {N: 4, Q: 0.9, PARENT: test_root, ACTIONS: {}}
test_root[ACTIONS][5] = {N: 2, Q: 0.1, PARENT: test_root, ACTIONS: {}}

print(select_action(test_root, CP))  # ==> 5 (0.8942 < 0.9965)
print(select_action(test_root, 0.3)) # ==> 3 (0.50895 > 0.45157)

5
3


In [14]:
# Algoritmul MCTS (UCT)
#  state0 - starea pentru care trebuie aleasă o acțiune
#  budget - numărul de iterații permis
#  tree - un arbore din explorările anterioare
#  opponent_s_action - ultima acțiune a adversarului

def mcts(state0, budget, tree, opponent_s_action = None):
    # DACĂ există un arbore construit anterior ȘI
    #   acesta are un copil ce corespunde ultimei acțiuni a adversarului,
    # ATUNCI acel copil va deveni nodul de început pentru algoritm.
    # ALTFEL, arborele de start este un nod gol.
    if tree and opponent_s_action in tree[ACTIONS]:
        tree = tree[ACTIONS][opponent_s_action]
    else:
        tree = init_node()
    
    #---------------------------------------------------------------
    for x in range(budget):
        # Punctul de start al simulării va fi rădăcina de start
        state = state0
        node = tree

        # Coborâm în arbore până când ajungem la o stare finală
        # sau la un nod cu acțiuni neexplorate.
        # Variabilele state și node se 'mută' împreună.
        while (not is_final(state)
            and all(action in node[ACTIONS] for action in get_available_actions(state))
        ):
            new_action = select_action(node)
            state = apply_action(state, new_action)
            node = node[ACTIONS][new_action]
        
        #---------------------------------------------------------------
        # Dacă am ajuns într-un nod care nu este final și din care nu s-au
        # `încercat` toate acțiunile, construim un nod nou.
        if not is_final(state):
            new_action = choice(list(filter(lambda a: a not in node[ACTIONS], get_available_actions(state))))

            state = apply_action(state, new_action)
            node = init_node(node)
            node[PARENT][ACTIONS][new_action] = node

        #---------------------------------------------------------------
        # Se simulează o desfășurare a jocului până la ajungerea într-o
        # starea finală. Se evaluează recompensa în acea stare.
        while not is_final(state):
            state = apply_action(state, choice(get_available_actions(state)))
        
        winner = is_final(state)
        if winner == state0[NEXT_PLAYER]:
            reward = 1
        elif winner == (3 - state0[NEXT_PLAYER]):
            reward = 0.0
        elif winner == 3:
            reward = 0.25
        else:
            reward = 0.5

        #---------------------------------------------------------------
        # Se actualizează toate nodurile de la node către rădăcină:
        #  - se incrementează valoarea N din fiecare nod
        #  - se adaugă recompensa la valoarea Q
        crt_node = node
        while crt_node:
            crt_node[N] += 1
            crt_node[Q] += reward
            crt_node = crt_node[PARENT]

        #---------------------------------------------------------------

    if tree:
        final_action = select_action(tree, 0.0)
        return (final_action, tree[ACTIONS][final_action])
    # Acest cod este aici doar ca să nu dea erori testele mai jos; în mod normal tree nu trebuie să fie None
    if get_available_actions(state0):
        return (get_available_actions(state0)[0], init_node())
    return (0, None)

In [15]:
# Testare MCTS
(action, tree) = mcts(init_state(), 20, None, None)
print(action)
if tree: print_tree(tree[PARENT])

3
N = 20, Q = 12.0
 0 => 
   N = 3, Q = 2.0
    1 => 
      N = 1, Q = 1
    5 => 
      N = 1, Q = 1
 3 => 
   N = 4, Q = 3.0
    6 => 
      N = 1, Q = 1
    4 => 
      N = 1, Q = 1
    2 => 
      N = 1, Q = 0.0
 2 => 
   N = 4, Q = 3.0
    1 => 
      N = 1, Q = 1
    3 => 
      N = 1, Q = 1
    6 => 
      N = 1, Q = 0.0
 4 => 
   N = 3, Q = 2.0
    3 => 
      N = 1, Q = 1
    0 => 
      N = 1, Q = 1
 5 => 
   N = 2, Q = 0.0
    4 => 
      N = 1, Q = 0.0
 6 => 
   N = 2, Q = 1.0
    2 => 
      N = 1, Q = 1
 1 => 
   N = 2, Q = 1.0
    6 => 
      N = 1, Q = 1


In [17]:
def play_games(games_no, budget1, budget2, verbose = False):
    # Efortul de căutare al jucătorilor
    budget = [budget1, budget2]
    
    score = {p: 0 for p in name}
        
    for i in range(games_no):
        # Memoriile inițiale
        memory = [None, None]
        
        # Se desfășoară jocul
        state = init_state()
        last_action = None
    
        while not is_final(state):
            p = state[NEXT_PLAYER] - 1
            (action, memory[p]) = mcts(state, budget[p], memory[p], last_action)
            state = apply_action(state, action)
            last_action = action
        
        # Cine a câștigat?
        winner = is_final(state)
        score[name[winner]] += + 1
        
        # Afișăm
        if verbose:
            print_state(state)
            if winner == 3: print("Remiză.")
            else: print(f"A câștigat {name[winner]}")

    # Afișează scorul final
    print(f"Scor final: {score}.")

In [18]:
# play_games(N, BR, BA, VERBOSE) - rulează N jocuri, cu bugetele BR pt ROȘU și BA pt ALBASTRU
play_games(5, 2, 30, True) # ne așteptăm să câștige ALBASTRU
play_games(5, 30, 2, True) # ne așteptăm să câștige ROȘU

|       |
|       |
|  R    |
|  RRA  |
|A AAR  |
|AARARRR|
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
|       |
|  A    |
|  RAA  |
|  ARAR |
|R ARAA |
|RRARRRA|
+-------+
Urmează: 1 - ROȘU
A câștigat ALBASTRU
| A    R|
| R    R|
| AA AAA|
| RR AAR|
|RAAARRR|
|RRAARRA|
+-------+
Urmează: 1 - ROȘU
A câștigat ALBASTRU
|       |
|       |
|  A    |
| RA    |
| AA R R|
|RAARARR|
+-------+
Urmează: 1 - ROȘU
A câștigat ALBASTRU
|       |
|       |
| A     |
| AR A  |
|RAR R  |
|RAR A  |
+-------+
Urmează: 1 - ROȘU
A câștigat ALBASTRU
Scor final: {'': 0, 'ROȘU': 1, 'ALBASTRU': 4, 'REMIZĂ': 0}.
|       |
|       |
|    A  |
|    A  |
|    A  |
|  RRRR |
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
|       |
|       |
|  R    |
|  R   R|
|  R A A|
|AAR A R|
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
|       |
|       |
| R     |
| RA  A |
| RR AA |
| RA RRA|
+-------+
Urmează: 2 - ALBASTRU
A câștigat ROȘU
|       |
|       |
|       |
|  A    |
|ARRRRAA|
|RRAAARR|
+-------+
Urmeaz